# Experiments on the Reproducibility of Results with LinTS
## Imports and Custom Class

In [2]:
# Run this cell for every experiment
import pandas as pd
import numpy as np
import platform
import pickle
from sklearn.preprocessing import StandardScaler

from mabwiser.mab import MAB
from mabwiser.linear import _RidgeRegression, _Linear

class LinTSExample(_RidgeRegression):
    def predict(self, x):
        beta_sampled = rng.multivariate_normal(x, self.A_inv)
        return np.dot(x, beta_sampled)
    
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)
 

For these experiments we are using the MovieLens 100k data set, available for download at [https://grouplens.org/datasets/movielens/](https://grouplens.org/datasets/movielens/)

Preprocessing is in the main notebook.

## Windows

In [4]:
platform.platform()

'Windows-10-10.0.18362-SP0'

In [7]:
import sklearn 
print(np.__version__)
print(sklearn.__version__)


1.19.4
0.23.0


In [8]:
users = pd.read_csv('users.csv')
responses = pd.read_csv('responses.csv')

In [9]:
train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

In [10]:
decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

In [11]:
rng = np.random.RandomState(seed=11)
item_ids = list(responses['item id'].unique())
mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [12]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
mab.fit(decisions, rewards, contexts)

In [14]:
win_expectations = mab.predict_expectations(test_contexts)

In [15]:
win_expectations[0][1]

13.999455973138756

In [16]:
win_expectations[0][2]

13.61917540941403

In [17]:
pickle.dump(mab, open('win_mab.pkl', 'wb'))
pickle.dump(win_expectations, open('win_expectations.pkl', 'wb'))